# CLEANING DATA

Authenticates to Space-Track, pulls GP_History for chosen satellites.

Cleans & validates types, de-dupes, sorts.

Derives semi-major axis, perigee/apogee altitudes from mean motion & eccentricity.

Builds daily series, rolling z-scores, and simple anomaly flags.

Exports a clean features table and an anomaly events CSV.

In [6]:
# run in a terminal cell once (or use requirements.txt)
!pip install requests python-dotenv pandas numpy sgp4 pandera ruptures matplotlib
!pip install --upgrade "pandas>=2.1.1" "pandera>=0.20.0"